In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten#create model
from keras.layers import Dropout, Activation
from keras.utils import to_categorical
from keras import losses

Using TensorFlow backend.


In [2]:
import fileinput
import numpy as np
filename = "turkish.txt" #wiki_00
trFile = open(filename ,'r+')

"**wiki_00**" file is from:  https://www.kaggle.com/mustfkeskin/turkish-wikipedia-dump

#### Data Preprocessing

In [3]:
def isChar(ch):
    if (ord(ch)>64 and ord(ch)<91) or (ord(ch)>96 and ord(ch)<123):
        return True
    else:
        return False
ord('/')    #Control
isChar('/')

False

In [4]:
def binValue(char):
    binArr =[]
    binStr = np.binary_repr(ord(char), width=8)
    
    for each in binStr:
         binArr.append(int(each,2))
    binArr = np.array(binArr)
    
    return binArr
binValue("A")

array([0, 1, 0, 0, 0, 0, 0, 1])

In [5]:
specialChar =['ö','ğ','ı','ü','ş','ç','İ','Ö','Ü','Ş','Ç']
engChar =['o','g','i','u','s','c','I','O','U','S','C']

engArr =[]
outArr =[]

for line in trFile:
    for a in line:
        if a in specialChar: 
            index = specialChar.index(a)
            b = engChar[index]
            engArr.append(binValue(b))
            outArr.append(1)
        elif not(isChar(a)):
            engArr.append(binValue(" "))
            outArr.append(0)
        else:               
            engArr.append(binValue(a))
            outArr.append(0)
                
engArr = np.array(engArr)
outArr = np.array(outArr)

In [6]:
engArr.shape

(2254, 8)

c =[]
c.append([1,2,3,4])
c.append([5,6,7,8])

c= np.array(c)
c.shape

***
**Results and Comments**
                                
- I changed all special characters in turkish file (trFile) and wrote with english characters to english file (engFile). And when I find special character I add to my array or output file 1 value else 0, to use those as my output in CNN. 

- Right now we have our input and output files ready. In the other part we should prepare CNN and let it learn from our input and output. I will use 8 bit ASCII value of each char in the input text (engFile).

* * *

|b|u|g|u|n|
|------|------|------|------|------|
|0|0|0|1|0|

- For example we have bugun written in engFile, we write output file 1 for turkish char "ü" as it can be seen above
***

In [7]:
sampleAmount=20#third dimention

eng2DArr =[]
k=0
for i in range(sampleAmount):
    eng2DArr.append(engArr[k:k+100])
    k+=100

eng2DArr = np.array(eng2DArr)
#eng2DArr = np.transpose(eng2DArr)
eng2DArr.shape

(20, 100, 8)

In [8]:
out2DArr =[]
k=0
for i in range(sampleAmount):
    out2DArr.append(outArr[k:k+100])
    k+=100
out2DArr = np.array(out2DArr)
out2DArr.shape

(20, 100)

In [9]:
x_train = np.expand_dims(eng2DArr , axis=4)
y_train = out2DArr

x_test = eng2DArr
y_test = out2DArr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  """Entry point for launching an IPython kernel.


In [10]:
print(x_train.shape)
print(y_train.shape)

(20, 100, 8, 1)
(20, 100)


In [11]:
model = Sequential()#add model layers
model.add(Conv2D(64, kernel_size=4, activation='relu', input_shape=(100,8,1)))
model.add(Dropout(0.25))
model.add(Conv2D(32, kernel_size=4, activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))

In [12]:
model.compile(optimizer='adam', loss=losses.binary_crossentropy, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 97, 5, 64)         1088      
_________________________________________________________________
dropout_1 (Dropout)          (None, 97, 5, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 94, 2, 32)         32800     
_________________________________________________________________
flatten_1 (Flatten)          (None, 6016)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               601700    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
Total params: 645,688
Trainable params: 645,688
Non-trainable params: 0
________________________________________________

In [13]:
model.fit(x_train, y_train, batch_size=100, epochs=50)

Epoch 1/50
20/20 [==============================] - 0s 10ms/step - loss: 0.7733 - accuracy: 0.9245
Epoch 2/50
20/20 [==============================] - 0s 2ms/step - loss: 0.7119 - accuracy: 0.8955
Epoch 3/50
20/20 [==============================] - 0s 2ms/step - loss: 0.7023 - accuracy: 0.8755
Epoch 4/50
20/20 [==============================] - 0s 2ms/step - loss: 0.6622 - accuracy: 0.8940
Epoch 5/50
20/20 [==============================] - 0s 2ms/step - loss: 0.6225 - accuracy: 0.9210
Epoch 6/50
20/20 [==============================] - 0s 2ms/step - loss: 0.6071 - accuracy: 0.9250
Epoch 7/50
20/20 [==============================] - 0s 2ms/step - loss: 0.5423 - accuracy: 0.9245
Epoch 8/50
20/20 [==============================] - 0s 2ms/step - loss: 0.5228 - accuracy: 0.9245
Epoch 9/50
20/20 [==============================] - 0s 2ms/step - loss: 0.5311 - accuracy: 0.9245
Epoch 10/50
20/20 [==============================] - 0s 3ms/step - loss: 0.4753 - accuracy: 0.9245
Epoch 11/50
20/20 

In [16]:
trFile.close()